<a href="https://colab.research.google.com/github/bbberylll/Summer_Project_ESAA_YB_2/blob/main/ESAA_%EC%86%90%ED%98%84%EA%B2%BD_%ED%94%BC%EC%B2%98%EB%B3%80%EA%B2%BD3_%EC%84%B1%EB%8A%A5_0_7814858415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y lightgbm
!apt-get update -qq
!apt-get install -y -qq cmake libboost-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM.git
%cd LightGBM

!mkdir build && cd build
!cmake -DUSE_GPU=ON ..

!make -j$(nproc)

%cd ../python-package
!python setup.py install --pre

Cloning into 'LightGBM'...
remote: Enumerating objects: 35473, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 35473 (delta 159), reused 83 (delta 83), pack-reused 35232 (from 2)
Receiving objects: 100% (35473/35473), 24.01 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (26375/26375), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21818, done.        
remote: Counting objects: 100% (45/45), done. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings


warnings.filterwarnings('ignore')


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ESAA/0_Summer_project/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ESAA/0_Summer_project/test.csv')

In [ ]:
train = train_df.copy() ## 데이터 복사
test = test_df.copy()

## 기존 피처 엔지니어링

In [ ]:
# payload 결측치 플래그
train['payload_bwd_missing_flag'] = train['payload_bwd_mean'].isna().astype(int)
test['payload_bwd_missing_flag'] = test['payload_bwd_mean'].isna().astype(int)

In [ ]:
# 포트 변수 변수 => -1
# payload, rate, iat_avg_packets, duration, => 중앙값


train['port_src'].fillna(-1, inplace=True) # -1 대체
train['port_dst'].fillna(-1, inplace=True)

payload_bwd_median = train['payload_bwd_mean'].median() # 중앙값 대체
train['payload_bwd_mean'].fillna(payload_bwd_median, inplace=True)

payload_fwd_median = train['payload_fwd_mean'].median()
train['payload_fwd_mean'].fillna(payload_fwd_median, inplace=True)

iat_median = train['iat_avg_packets'].median()
train['iat_avg_packets'].fillna(iat_median, inplace=True)

duration_median = train['duration'].median()
train['duration'].fillna(duration_median, inplace=True)

rate_fwd_median = train['rate_fwd_pkts'].median()
train['rate_fwd_pkts'].fillna(rate_fwd_median, inplace=True)

rate_bwd_median = train['rate_bwd_pkts'].median()
train['rate_bwd_pkts'].fillna(rate_bwd_median, inplace=True)

In [ ]:
test['port_src'].fillna(-1, inplace=True) # -1 대체
test['port_dst'].fillna(-1, inplace=True)

payload_bwd_median = test['payload_bwd_mean'].median() # 중앙값 대체
test['payload_bwd_mean'].fillna(payload_bwd_median, inplace=True)

payload_fwd_median = test['payload_fwd_mean'].median()
test['payload_fwd_mean'].fillna(payload_fwd_median, inplace=True)

iat_median = test['iat_avg_packets'].median()
test['iat_avg_packets'].fillna(iat_median, inplace=True)

duration_median = test['duration'].median()
test['duration'].fillna(duration_median, inplace=True)

rate_fwd_median = test['rate_fwd_pkts'].median()
test['rate_fwd_pkts'].fillna(rate_fwd_median, inplace=True)

rate_bwd_median = test['rate_bwd_pkts'].median()
test['rate_bwd_pkts'].fillna(rate_bwd_median, inplace=True)

In [ ]:
#=============================================================================
# tcp_syn_count => 3개로 범주화
# port_dst => 1~2023/이후 상위 5개/나머지
#=============================================================================

def categorize_syn(x):
    if x == 0:
        return 0  # 정상
    elif x in [1, 2]:
        return 1  # 약간의 SYN 요청
    else:
        return 2  # SYN Flooding 가능성

train['syn_flag_cat'] = train['tcp_syn_count'].apply(categorize_syn)
test['syn_flag_cat'] = test['tcp_syn_count'].apply(categorize_syn)

In [ ]:
# port_dst>1023 데이터 중 상위 5개 포트 추출
dst_port_high = train[train['port_dst'] > 1023]['port_dst']
top5_ports = dst_port_high.value_counts().nlargest(5).index

def categorize_port_dst(p):
    if p <= 1023:
        return str(int(p))  # 0~1023은 그대로
    elif p in top5_ports:
        return f'special_{int(p)}' # 1024, 8072, 49152, 8079, 2602
    else:
        return 'other'

train['port_dst_cat'] = train['port_dst'].apply(categorize_port_dst)

In [ ]:

dst_port_high_test = test[test['port_dst'] > 1023]['port_dst']
top5_ports_test = dst_port_high_test.value_counts().nlargest(5).index

def categorize_port_dst_test(p):
    if p <= 1023:
        return str(int(p))  # 0~1023은 그대로
    elif p in top5_ports_test:
        return f'special_{int(p)}' # 1024, 8072, 49152, 8079, 2602
    else:
        return 'other'

test['port_dst_cat'] = test['port_dst'].apply(categorize_port_dst_test)

In [ ]:
# PSH 유무: tcp_psh_count > 0 → 1, else → 0
#train['has_psh_flag'] = (train['tcp_psh_count'] > 0).astype(int)
#test['has_psh_flag'] = (test['tcp_psh_count'] > 0).astype(int)

# 초당 패킷 전송 비율 = rate_fwd_pkts / rate_bwd_pkts
train['rate_pkts_ratio'] = train['rate_fwd_pkts'] / (train['rate_bwd_pkts'] + 1e-5) # 0으로 나누는 것 방지
test['rate_pkts_ratio'] = test['rate_fwd_pkts'] / (test['rate_bwd_pkts'] + 1e-5)

# 송수신 패킷 비율 = pkt_count_fwd / pkt_count_bwd
train['pkt_count_ratio'] = train['pkt_count_fwd'] / (train['pkt_count_bwd'] + 1e-5)
test['pkt_count_ratio'] = test['pkt_count_fwd'] / (test['pkt_count_bwd'] + 1e-5)

# 패킷 간 평균 시간 간격 이진 파생변수: iat_avg_packets <= 1 → 1, else → 0
#train['is_iat_low'] = (train['iat_avg_packets'] <= 1.0).astype(int)
#test['is_iat_low'] = (test['iat_avg_packets'] <= 1.0).astype(int)

# 포트 클러스터링 기반 파생변수 추가 (KMeans)
from sklearn.cluster import KMeans

port_dst_train = train_df[['port_dst']].fillna(-1)
port_dst_test = test_df[['port_dst']].fillna(-1)

kmeans = KMeans(n_clusters=6, random_state=156)
train['port_dst_cluster'] = kmeans.fit_predict(port_dst_train)
test['port_dst_cluster'] = kmeans.predict(port_dst_test)

# 비율 기반 파생변수 추가 (payload / (payload +duration))
train['payload_ratio'] = train['payload_bwd_mean'] / (train['payload_bwd_mean'] + train['duration'] + 1e-5)
test['payload_ratio'] = test['payload_bwd_mean'] / (test['payload_bwd_mean'] + test['duration'] + 1e-5)

# TCP 윈도우 비율 파생변수 추기
train['win_ratio'] = train['tcp_win_fwd_init'] / (train['tcp_win_bwd_init'] + 1e-5)
test['win_ratio'] = test['tcp_win_fwd_init'] / (test['tcp_win_bwd_init'] + 1e-5)


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        11999 non-null  object 
 1   ip_src                    9487 non-null   object 
 2   port_src                  11999 non-null  float64
 3   ip_dst                    10695 non-null  object 
 4   port_dst                  11999 non-null  float64
 5   protocol                  11999 non-null  object 
 6   duration                  11999 non-null  float64
 7   pkt_count_fwd             11999 non-null  int64  
 8   pkt_count_bwd             11999 non-null  int64  
 9   rate_fwd_pkts             11999 non-null  float64
 10  rate_bwd_pkts             11999 non-null  float64
 11  rate_fwd_bytes            11999 non-null  float64
 12  rate_bwd_bytes            11999 non-null  float64
 13  payload_fwd_mean          11999 non-null  float64
 14  payloa

# 새로운 피처 엔지니어링
## 성능 향상

In [ ]:
from scipy.stats.mstats import winsorize

# 세션 특성 처리
# a) iat_avg_packets: 윈저라이징(상·하위 1%)
train['iat_avg_packets_clipped'] = winsorize(train['iat_avg_packets'], limits=(0.01, 0.01))
test['iat_avg_packets_clipped'] = winsorize(test['iat_avg_packets'], limits=(0.01, 0.01))

# duration: 이미 'duration'에 대해 윈저라이징/스케일링이 필요시
# 윈저라이징 (상·하위 1%)
train['duration_clipped'] = winsorize(train['duration'], limits=(0.01, 0.01))
test['duration_clipped'] = winsorize(test['duration'], limits=(0.01, 0.01))


## try 해보는 거

In [ ]:
epsilon = 1e-6  # 분모가 0일 때 안정성 확보

# interaction term (duration × pkt_count_fwd)
train['duration_pkt_fwd_interaction'] = train['duration'] * train['pkt_count_fwd']
test['duration_pkt_fwd_interaction'] = test['duration'] * test['pkt_count_fwd']


train['rate_bytes_diff'] = train['rate_fwd_bytes'] - train['rate_bwd_bytes']
train['pkt_count_ratio'] = train['pkt_count_fwd'] / (train['pkt_count_bwd'] + epsilon)
train['duration_rate_fwd_pkts_interaction'] = train['duration'] * train['rate_fwd_pkts']

test['rate_bytes_diff'] = test['rate_fwd_bytes'] - test['rate_bwd_bytes']
test['pkt_count_ratio'] = test['pkt_count_fwd'] / (test['pkt_count_bwd'] + epsilon)
test['duration_rate_fwd_pkts_interaction'] = test['duration'] * test['rate_fwd_pkts']


## 기존 코드 2

In [ ]:
train.drop(['tcp_psh_count','rate_fwd_pkts','rate_bwd_pkts',
            'pkt_count_fwd','pkt_count_bwd'], axis=1, inplace=True) ## 불필요한 기존변수삭제
test.drop(['tcp_psh_count','rate_fwd_pkts','rate_bwd_pkts',
            'pkt_count_fwd','pkt_count_bwd'], axis=1, inplace=True)

In [ ]:
# ID, ip_scr, ip_dst, port_src
# payload_fwd_mean (bwd와 겹치는 칼럼)

train.drop(['ID','ip_src','ip_dst','payload_fwd_mean','port_src','port_dst','tcp_syn_count'], axis=1, inplace=True)
test.drop(['ID','ip_src','ip_dst','payload_fwd_mean','port_src','port_dst','tcp_syn_count'], axis=1, inplace=True)

In [ ]:
import numpy as np

log_cols = [
    'duration',
    'rate_fwd_bytes',
    'rate_bwd_bytes',
    'payload_bwd_mean',
    'tcp_rst_count',
    'tcp_win_fwd_init',
    'tcp_win_bwd_init',
    'iat_avg_packets',
    'rate_pkts_ratio',
    'pkt_count_ratio',
    'payload_ratio',
    'win_ratio'
]

for col in log_cols:
    train[col] = np.log1p(train[col])
    test[col] = np.log1p(test[col])

In [ ]:
from sklearn.preprocessing import StandardScaler

scale_cols = log_cols.copy()

# 스케일러 객체 생성
scaler = StandardScaler()

# 스케일링 적용
train[scale_cols] = scaler.fit_transform(train[scale_cols])
test[scale_cols] = scaler.transform(test[scale_cols])

In [ ]:

from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
le_protocol = LabelEncoder()
le_postdst = LabelEncoder()
le_attack = LabelEncoder()

le_protocol.fit(pd.concat([train['protocol'], test['protocol']], axis=0).astype(str).unique())
le_postdst.fit(pd.concat([train['port_dst_cat'], test['port_dst_cat']], axis=0).astype(str).unique())

train['protocol'] = le_protocol.transform(train['protocol'].astype(str))
train['port_dst_cat'] = le_postdst.transform(train['port_dst_cat'].astype(str))
train['attack_type'] = le_attack.fit_transform(train['attack_type']) # attack_type is only in train

test['protocol'] = le_protocol.transform(test['protocol'].astype(str))
test['port_dst_cat'] = le_postdst.transform(test['port_dst_cat'].astype(str))

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   protocol                            11999 non-null  int64  
 1   duration                            11999 non-null  float64
 2   rate_fwd_bytes                      11999 non-null  float64
 3   rate_bwd_bytes                      11999 non-null  float64
 4   payload_bwd_mean                    11999 non-null  float64
 5   tcp_win_fwd_init                    11999 non-null  float64
 6   tcp_win_bwd_init                    11999 non-null  float64
 7   tcp_rst_count                       11999 non-null  float64
 8   iat_avg_packets                     11999 non-null  float64
 9   attack_type                         11999 non-null  int64  
 10  payload_bwd_missing_flag            11999 non-null  int64  
 11  syn_flag_cat                        11999

In [ ]:
for col in train.select_dtypes(include='category').columns:
    train[col] = train[col].cat.codes
    test[col]  = test[col].cat.codes

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   protocol                            11999 non-null  int64  
 1   duration                            11999 non-null  float64
 2   rate_fwd_bytes                      11999 non-null  float64
 3   rate_bwd_bytes                      11999 non-null  float64
 4   payload_bwd_mean                    11999 non-null  float64
 5   tcp_win_fwd_init                    11999 non-null  float64
 6   tcp_win_bwd_init                    11999 non-null  float64
 7   tcp_rst_count                       11999 non-null  float64
 8   iat_avg_packets                     11999 non-null  float64
 9   attack_type                         11999 non-null  int64  
 10  payload_bwd_missing_flag            11999 non-null  int64  
 11  syn_flag_cat                        11999

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   protocol                            3000 non-null   int64  
 1   duration                            3000 non-null   float64
 2   rate_fwd_bytes                      3000 non-null   float64
 3   rate_bwd_bytes                      3000 non-null   float64
 4   payload_bwd_mean                    3000 non-null   float64
 5   tcp_win_fwd_init                    3000 non-null   float64
 6   tcp_win_bwd_init                    3000 non-null   float64
 7   tcp_rst_count                       3000 non-null   float64
 8   iat_avg_packets                     3000 non-null   float64
 9   payload_bwd_missing_flag            3000 non-null   int64  
 10  syn_flag_cat                        3000 non-null   int64  
 11  port_dst_cat                        3000 no

## 방향성

1. SMOTE 미적용, SMOTE만 적용, SMOTE+언더 3가지 케이스 실험
2. 데이터 엔지니어링 미변경 / 변경
3. Voting을 해야 함


## SMOTE 미적용
### LGBM

In [ ]:
cp LightGBM/build/lib_lightgbm.so LightGBM/python-package/lightgbm/lib/
python setup.py install

SyntaxError: invalid syntax (ipython-input-52-2789070033.py, line 1)

In [ ]:
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split

Exception: Cannot find lightgbm library file in following paths:
/content/LightGBM/python-package/lib_lightgbm.so
/content/LightGBM/python-package/lightgbm/bin/lib_lightgbm.so
/content/LightGBM/python-package/lightgbm/lib/lib_lightgbm.so

In [ ]:
X_1 = train.drop(['attack_type'], axis=1)
y_1 = train['attack_type']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( X_1, y_1, test_size=0.2, random_state=156, stratify=y_1)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_1)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_1)
class_weight_dict = dict(zip(classes, weights))

In [ ]:
lgbm_clf = lgb.LGBMClassifier(class_weight=class_weight_dict, random_state=156,
                              boosting_type='gbdt',
                              n_estimators=1000,
                              learning_rate=0.05,
                              device='gpu',
                              gpu_platform_id=0,
                              gpu_device_id=0)

lgbm_clf.fit(X_train, y_train)


In [ ]:
y_pred_1 = lgbm_clf.predict(X_val)

In [ ]:
params = lgbm_clf.get_params()
for k, v in params.items():
    print(f"{k}: {v}")

In [ ]:
test_pred = lgbm_clf.predict(test)

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import lightgbm as lgb
import numpy as np
import pandas as pd

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scores = cross_val_score(
    estimator=lgbm_clf,
    X=X_1, y=y_1,
    cv=skf,
    scoring='f1_macro',
    n_jobs=-1
)

print(f'Fold별 F1‑macro 점수: {scores}')
print(f'평균 F1‑macro: {scores.mean():.4f} ± {scores.std():.4f}')

Fold별 F1‑macro 점수: [0.77123883 0.79382709 0.82709204 0.75213666 0.77821325]
평균 F1‑macro: 0.7845 ± 0.0252


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA/0_Summer_project/sample_submission.csv')

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'attack_type': test_pred
})

submission['attack_type'] = le_attack.inverse_transform(submission['attack_type'])


submission.to_csv('submission_lgbm_noSMOTE_featuring5.csv', index=False)


In [ ]:
from google.colab import files
files.download('submission_lgbm_noSMOTE_featuring5.csv')

리더보드 점수 : 0.7814858415

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ftr_importances_values = lgbm_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=X_1.columns)
ftr_top = ftr_importances.sort_values(ascending=False)

# 피처 중요도 시각화
plt.figure(figsize=(8,6))
plt.title('Feature importances')
sns.barplot(x=ftr_top, y=ftr_top.index)
plt.show()

# 하이퍼 파라미터

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
import lightgbm as lgb
import numpy as np

In [ ]:
base_params = lgbm_clf.get_params()

In [ ]:
param_grid_expanded = {
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
    'num_leaves': [15, 31, 63, 127, None],
    'max_depth': [4, 6, 8, 10, -1, None],
    'subsample': [0.5, 0.7, 0.8, 1.0, None],
    'colsample_bytree': [0.5, 0.7, 0.8, 1.0, None],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, None],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, None],
    'min_child_samples': [5, 10, 20, 50, None],
    'min_child_weight': [0.001, 0.01, 0.1, None],
    'boosting_type': ['gbdt', 'dart', 'goss', None],
    'min_split_gain': [0.0, 0.1, 0.2, None],
    'n_estimators': [100, 300, 500, 1000]
}

gpu_params = base_params.copy()
gpu_params.update({
    'device': 'gpu',            # GPU 사용
    'gpu_platform_id': 0,       # 필요 시
    'gpu_device_id': 0          # 필요 시
})

In [ ]:
model = lgb.LGBMClassifier(**gpu_params)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=156)

In [ ]:
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid_expanded,
    scoring='f1_macro',
    cv=skf,
    n_jobs=-1,
    verbose=2
)
grid.fit(X_1, y_1)

In [ ]:
print("▶ 최적 파라미터:", grid.best_params_)
print(f"▶ 최고 CV F1‑macro: {grid.best_score_:.4f}")

In [ ]:
!nvidia-smi


Wed Jul 23 07:17:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, make_scorer
from imblearn.over_sampling import SMOTE

In [ ]:
# 타겟 및 피처 나누기
X_1 = train.drop(['attack_type'], axis=1)
y_1 = train['attack_type']

# 학습/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split( X_1, y_1, test_size=0.2, random_state=156, stratify=y_1)


In [ ]:
rf_model = RandomForestClassifier(
    random_state=156,
    n_estimators=100,
    n_jobs=-1
)

In [ ]:
rf_model.fit(X_1, y_1)
test_pred = rf_model.predict(test)


### XGBoost

In [ ]:
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [ ]:
X_1 = train.drop(['attack_type'], axis=1)
y_1 = train['attack_type']

X_train, X_val, y_train, y_val = train_test_split(
    X_1, y_1, test_size=0.2, stratify=y_1, random_state=156)

In [ ]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights_dict = dict(zip(classes, weights))

In [ ]:
xgb_clf = XGBClassifier(random_state=156, n_jobs=-1, use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_1, y_1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=-1, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
test_pred = xgb_clf.predict(test)

In [ ]:
params_2 = xgb_clf.get_params()
for k, v in params_2.items():
    print(f"{k}: {v}")

objective: multi:softprob
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
device: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: mlogloss
feature_types: None
gamma: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: None
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: None
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
multi_strategy: None
n_estimators: None
n_jobs: -1
num_parallel_tree: None
random_state: 156
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None
use_label_encoder: False


In [ ]:
lgbm_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    class_weight={
        0: 0.11374322223486141,
        1: 37.03395061728395,
        2: 2.122965322009908,
        3: 21.27482269503546,
        4: 24.38821138211382,
        5: 0.5816850882295909,
        6: 1.260928961748634,
        7: 33.330555555555556,
        8: 29.409313725490197,
        9: 38.458333333333336,
        10: 71.42261904761905,
        11: 166.65277777777777
    },
    colsample_bytree=1.0,
    importance_type='split',
    learning_rate=0.1,
    max_depth=-1,
    min_child_samples=20,
    min_child_weight=0.001,
    min_split_gain=0.0,
    n_estimators=100,
    num_leaves=31,
    random_state=156,
    reg_alpha=0.0,
    reg_lambda=0.0,
    subsample=1.0,
    subsample_for_bin=200000,
    subsample_freq=0
)

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=156,
    n_jobs=-1
)

xgb_model = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=156,
    n_jobs=-1
)

### Voting 1


In [ ]:
from sklearn.ensemble import VotingClassifier

voting1 = VotingClassifier(
    estimators=[
        ('lgbm', lgbm_model),
        ('rf',   rf_model),
        ('xgb',  xgb_model)
    ],
    voting='soft',
    weights=[0.7623, 0.6804, 0.6781],
    n_jobs=-1
)

In [ ]:
voting1.fit(X_train, y_train)


VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(class_weight={0: 0.11374322223486141,
                                                           1: 37.03395061728395,
                                                           2: 2.122965322009908,
                                                           3: 21.27482269503546,
                                                           4: 24.38821138211382,
                                                           5: 0.5816850882295909,
                                                           6: 1.260928961748634,
                                                           7: 33.330555555555556,
                                                           8: 29.409313725490197,
                                                           9: 38.458333333333336,
                                                           10: 71.42261904761905,
                                                           11: 166.65277777777777},
                                             random_state=156)),
                             ('rf',
                              RandomFor...
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=-1,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...))],
                 n_jobs=-1, voting='soft', weights=[0.7623, 0.6804, 0.6781])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

y_pred = voting1.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1_macro = f1_score(y_val, y_pred, average='macro')


In [ ]:
print(f'Voting 모델 Accuracy: {accuracy:.4f}')
print(f'Voting 모델 Macro F1-score: {f1_macro:.4f}')

print(classification_report(y_val, y_pred))

conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Voting 모델 Accuracy: 0.9875
Voting 모델 Macro F1-score: 0.8359
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1758
           1       1.00      0.80      0.89         5
           2       1.00      0.99      0.99        94
           3       0.88      0.70      0.78        10
           4       0.70      0.88      0.78         8
           5       0.97      0.99      0.98       344
           6       0.99      0.99      0.99       159
           7       1.00      1.00      1.00         6
           8       0.75      0.43      0.55         7
           9       1.00      0.60      0.75         5
          10       0.67      0.67      0.67         3
          11       0.50      1.00      0.67         1

    accuracy                           0.99      2400
   macro avg       0.87      0.84      0.84      2400
weighted avg       0.99      0.99      0.99      2400

Confusion Matrix:
[[1746    0    0    1    0    7    1    0    1    0    

In [ ]:
voting1.fit(X_1, y_1)
test_pred = voting1.predict(test)

sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA/0_Summer_project/sample_submission.csv')

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'attack_type': test_pred
})

submission['attack_type'] = le_attack.inverse_transform(submission['attack_type'])


submission.to_csv('submission_lgbm_voting1.csv', index=False)


In [ ]:
from google.colab import files
files.download('submission_lgbm_voting1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Voting 2

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
cb_params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6
}

In [ ]:
cb   = CatBoostClassifier(**cb_params, verbose=False, random_state=156)

vot2 = VotingClassifier(
    estimators=[('lgbm', lgbm_model), ('xgb', xgb_model), ('cb', cb)],
    voting='soft',
    weights=[0.5, 0.3, 0.2]
)
vot2.fit(X_1, y_1)
vot2.predict(test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 11999, number of used features: 16
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907

array([0, 6, 0, ..., 0, 0, 0])

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'attack_type': test_pred
})

submission['attack_type'] = le_attack.inverse_transform(submission['attack_type'])


submission.to_csv('submission_lgbm_voting2.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission_lgbm_voting2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Voting 3

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier

et  = ExtraTreesClassifier(n_estimators=200, random_state=42)

vot3 = VotingClassifier(
    estimators=[('lgbm', lgbm_model), ('rf', rf_model), ('et', et)],
    voting='soft',
    weights=[0.6, 0.2, 0.2]
)
vot3.fit(X_1, y_1)
vot3.predict(test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 11999, number of used features: 16
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907

array([0, 6, 0, ..., 0, 0, 0])

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'attack_type': test_pred
})

submission['attack_type'] = le_attack.inverse_transform(submission['attack_type'])


submission.to_csv('submission_lgbm_voting3.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission_lgbm_voting3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Voting 4

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

svc  = SVC(probability=True, kernel='rbf', C=1.0, random_state=156)
lr   = LogisticRegression(max_iter=1000, C=1.0, random_state=156)
mlp  = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=156)

vot4 = VotingClassifier(
    estimators=[('lgbm', lgbm_model), ('svc', svc), ('lr', lr), ('mlp', mlp)],
    voting='soft'
)
vot4.fit(X_1, y_1)
vot4.predict(test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003556 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 11999, number of used features: 16
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Info] Start training from score -2.484907
[LightGBM] [Warning] No further splits with positive gain, best

array([0, 6, 0, ..., 0, 0, 0])

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'attack_type': test_pred
})

submission['attack_type'] = le_attack.inverse_transform(submission['attack_type'])


submission.to_csv('submission_lgbm_voting4.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission_lgbm_voting4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## SMOTE만 적용

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=156, k_neighbors=3)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
X_2 = train.drop(['attack_type'], axis=1)
y_2 = train['attack_type']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_2, y_2, test_size=0.3, stratify=y, random_state=156
)

In [ ]:
smote = SMOTE(k_neighbors=3, random_state=156)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

## SMOTE+언더

# Voting 완전히 새로 진행